I'm gonna  just sanity check the performance of the KNN-CDF computation I implemented. 

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from pearce.mocks.kittens import TrainingBox
import numpy as np
from copy import deepcopy
from halotools.utils.table_utils import compute_conditional_percentiles
from halotools.mock_observables import get_haloprop_of_galaxies
from pearce.mocks.void_density_function import knn_cdf
from matplotlib.colors import ListedColormap

In [3]:
boxno = 20

In [4]:
cat = TrainingBox(boxno, system = 'ki-ls')

In [5]:
cat.load(1.0, HOD='zheng07', biased_satellites=True)

/u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/satellites/nfw/biased_nfw_phase_space.py:197: UserWarning: You have selected 99 bins to digitize host halo concentration 
and 11 bins to digitize the galaxy bias parameter.
To populate mocks, the BiasedNFWPhaseSpace class builds a lookup table with shape (99, 11, 101),
one entry for every numerical solution to the Jeans equation.
Using this fine of a binning requires a long pre-computation of 109989 integrals
.Make sure you actually need to use so many bins
  warn(lookup_table_performance_warning.format(*args))
/u/ki/swmclau2/.conda/envs/hodemulator/lib/python2.7/site-packages/halotools/empirical_models/phase_space_models/analytic_models/satellites/nfw/biased_nfw_phase_space.py:197: UserWarning: You have selected 99 bins to digitize host halo concentration 
and 11 bins to digitize the galaxy bias parameter.
To populate mocks, the BiasedNFWPhaseSpace class bui

In [6]:
r_bins = np.logspace(-0.3, 2, 21)
rbins = r_bins
rbc = (r_bins[1:]+r_bins[:-1])/2.0

In [7]:
mass_bin_range = (12,16)
mass_bin_size = 0.1
mass_bins = np.logspace(mass_bin_range[0], mass_bin_range[1],
                           int((mass_bin_range[1] - mass_bin_range[0]) / mass_bin_size) + 1)
mass_bin_centers = (mass_bins[:-1] + mass_bins[1:]) / 2

In [8]:
k=2

In [9]:
def peaked_cdf(cdf):
    cdf[cdf>0.5] = 1-cdf[cdf>0.5]
    return cdf

In [10]:
def split_hod_knn_cdf_plot(HOD, ab_params,sec_haloprop_key,ks=[1,2,4,8], cmap_name = 'BrBG'):
    cat.load_model(1.0, HOD=HOD, hod_kwargs= {'sec_haloprop_key': sec_haloprop_key})
    cat.model.param_dict['logMmin'] = 13.5
    cat.model.param_dict['logM0'] = 12.5
    n_splits = len(ks)
    cat.populate(ab_params, min_ptcl = 20)
    #print cat.model.param_dict
    catalog = cat.model.mock.galaxy_table
    #sec_percentiles = compute_conditional_percentiles(prim_haloprop = cat.model.mock.halo_table['halo_mvir'],\
    #                                              sec_haloprop = cat.model.mock.halo_table[sec_haloprop_key],
    #                                          prim_haloprop_bin_boundaries= mass_bins)
    
    #sec_gal_percentiles = get_haloprop_of_galaxies(catalog['halo_id'], cat.model.mock.halo_table['halo_id'],
    #                                           sec_percentiles)
    
    # TODO bins here
    vdfs = np.zeros((n_splits, len(rbc)))
    #perc_ranges = np.linspace(0,1, n_splits+1)
    
    #cmap = [ListedColormap(c) for c in sns.color_palette(cmap_name, n_splits)]
    cmap = sns.color_palette(cmap_name, n_splits)

    #cmap = sns.dark_palette(cmap_name, n_splits)
    #gal_pos = np.c_[[catalog[_c] for _c in ['x','y','z']]].T

    #base_vdf = void_density_function(gal_pos, rbins, n_ran=int(1e7), period = cat.Lbox, n_jobs = 4, PBC=True)
    #plt.plot(rbc, base_vdf, c = 'k', label = 'All')

    for i,(c,k) in enumerate(zip(cmap,ks)):
        #sec_bin_gals = np.logical_and(perc_ranges[i] < sec_gal_percentiles, sec_gal_percentiles<perc_ranges[i+1])
        
        gal_pos = np.c_[[catalog[_c] for _c in ['x','y','z']]].T

        vdfs[i,:] = knn_cdf(gal_pos, rbins, k=k, n_ran=int(1e7), period = cat.Lbox, n_jobs = 4, PBC=True)
        #print c
        vdfs[i] = peaked_cdf(vdfs[i])
        plt.plot(rbc, vdfs[i], c = c, label = 'k=%d'%k)

    
    #gal_hist, _ = np.histogram(catalog['halo_mvir'], bins = mass_bins)
    #halo_hist, _= np.histogram(cat.model.mock.halo_table['halo_mvir'], bins = mass_bins)
    #full_hod = gal_hist*1.0/halo_hist
    
    
    #plt.plot(mass_bin_centers, full_hod, label = 'Full HOD', color = 'k')
    plt.legend(loc='best')
    plt.xscale('log');
    #plt.xscale('log')
    #plt.xlim(1e12,5e14)
    #plt.ylim([0, 40])
    plt.xlabel(r"$r$ [Mpc/h]")
    plt.ylabel(r"KNN_CDF K=%d"%k)
    plt.title(sec_haloprop_key)
    plt.show()

In [ ]:
ab_params = {'mean_occupation_centrals_assembias_corr1': 0.0,
             'mean_occupation_satellites_assembias_corr1': -1.0}

In [ ]:
split_hod_knn_cdf_plot('corrZheng07', ab_params, 'halo_local_density_10')

In [ ]:
fiducial_hod = {'log_Mmin': 13.0, 'sigma_logM': 0.2, 'logM0': 12.5, 'logM1': 14.0, 'alpha': 1.05}

In [ ]:
cat.populate(fiducial_hod)

In [ ]:
cat.calc_analytic_nd()

In [ ]:
fiducial_vdf = cat.calc_knn_cdf(r_bins,k=1, n_ran=1e6)

In [ ]:
plt.plot(rbc, fiducial_vdf)
plt.loglog();

In [ ]:
# measure shot noise
N = 10
vdfs = np.zeros((N, rbc.shape[0]))
for i in xrange(N):
    vdfs[i] = cat.calc_knn_cdf(r_bins, n_ran =1e6,k=k)

In [ ]:
plt.errorbar(rbc, vdfs.mean(axis=0), yerr = vdfs.std(axis=0) )
plt.loglog();

In [ ]:
plt.plot(rbc, vdfs.mean(axis=0)/vdfs.std(axis=0))
plt.loglog();

In [ ]:
#np.save('vdfs_nran100.npy', vdfs)
#np.save('vdfs_nran10.npy', vdfs)

In [ ]:
varied_pname = 'alpha'
low, high = 0.8, 1.2 
N = 6
cmap = sns.cubehelix_palette(N, rot=-.4, as_cmap=False)
vals = np.linspace(low, high, N)

In [ ]:
sns.palplot(cmap)

In [ ]:
plot_vdfs = np.zeros((N, len(rbc)))
for i,(c,v) in enumerate(zip(cmap, vals)):
    print v
    params = deepcopy(fiducial_hod)
    params[varied_pname] = v
    cat.populate(params)
    plot_vdfs[i] = cat.calc_knn_cdf(r_bins, n_ran=1e6, k =k )

In [ ]:
for c, pv, val in zip(cmap, plot_vdfs, vals):
    plt.plot(rbc, pv/fiducial_vdf, color = c, label=val)
    
plt.legend(loc='best')
plt.title(varied_pname)    
plt.xscale('log');

In [ ]:
# do random HODs
bounds_dict = {#'H0': (61.69472, 74.76751999999999),
 #'Neff': (2.62125, 4.27875),
 'alpha': (0.7, 1.3),
 'conc_gal_bias': (0.5, 2.0),
 #'sigma8': (0.65, 1.0), # TODO update
 'logM0': (12.6, 13.6),
 'logM1': (13.7, 14.7),
#'ns': (0.9278462, 0.9974495999999999),
# 'ombh2': (0.02066455, 0.02371239),
# 'omch2': (0.1012181, 0.13177679999999997),
 'sigma_logM': (0.05, 0.6)}
# 'ln10As': (3.0, 3.1),
# 'w0': (-1.399921, -0.5658486),
# 'Om': (0.25, 0.35),
# 'S8': (0.7, 1.0),
 #'Om': (0.15, 0.4),
 #'S8': (0.5, 1.1),
# 'mean_occupation_centrals_assembias_corr1': (-1.0, 1.0),
# 'mean_occupation_satellites_assembias_corr1': (-1.0, 1.0),
# 'mean_occupation_centrals_assembias_param1': (-1.0, 1.0),
# 'mean_occupation_satellites_assembias_param1': (-1.0, 1.0),
# 'mean_occupation_centrals_assembias_slope1': (-3.0, 3.0),
# 'mean_occupation_satellites_assembias_slope1': (-3.0, 3.0)}

In [ ]:
N = 10

for i in xrange(N):
    point = {}
    for key, bounds in bounds_dict.iteritems():
        point[key] = np.random.uniform(bounds[0], bounds[1])
        
    cat.populate(point)
    cat._add_logMmin(5e-4, 0)
    cat.populate()
    
    val = cat.calc_knn_cdf(r_bins, n_ran=1e8, k=8)
    plt.plot(rbc, peaked_cdf(val), alpha = 0.3)
    
plt.loglog()
plt.ylim([1e-4, 1.0])
plt.show()